In [88]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras as kr
import random

## Generator class

In [111]:
class generator():
    def __init__(self):
        self.char_to_int = {}
        self.int_to_char = []

        count = 0

        # Small letters
        count = self.add_char("a",26,count)

        # Capital letters
        count = self.add_char("A",26,count)

        # Decimal values
        count = self.add_char("0",10,count)
        
        # Special characters
        count = self.add_char(".",1,count)
        count = self.add_char("-",1,count)
        count = self.add_char(" ",1,count)
        
        print("char_to_int: ", self.char_to_int)
        print("int_to_char: ",self.int_to_char)
     
    def add_char(self,startChar,num_run,count):
        for i in range(num_run):
            char = chr(ord(startChar)+i)
            self.char_to_int[char] = count
            self.int_to_char.append(char)
            count += 1
        return count
    
    def add_spaces(self,cur_string,cur_x,cur_y,min_val,max_val):
        num_spaces = min_val + int(random.random()*max_val)

        for j in range(num_spaces):
            char = " "
            cur_string  += char
            cur_x.append(self.char_to_int[char])
            cur_y.append(0)
        return cur_string,cur_x,cur_y

    def add_date(self,cur_string,cur_x,cur_y):
        month = 1+int(random.random()*12.0)
        day = 1+int(random.random()*30.0)

        date_string = str(month) + "-" + str(day)

        for j in range(len(date_string)):
            char = date_string[j]
            cur_string += char
            cur_x.append(self.char_to_int[char])
            cur_y.append(1)
        return cur_string,cur_x,cur_y

    def random_discription(self,cur_string,cur_x,cur_y,disc_len):
        for j in range(disc_len):
            rand_int = int(random.random()*len(self.char_to_int))
            char = self.int_to_char[rand_int]
            cur_string  += char
            cur_x.append(self.char_to_int[char])
            cur_y.append(2)

        return cur_string,cur_x,cur_y

    def add_rand_value(self,cur_string,category,cur_x,cur_y):

        char = "R"
        cur_string  += char
        cur_x.append(self.char_to_int[char])
        cur_y.append(category)

        rand_float = round(random.random()*1000.0,2)

        value_as_string = str(rand_float)

        for j in range(len(value_as_string)):
            char = value_as_string[j]
            cur_string  += char
            cur_x.append(self.char_to_int[char])
            cur_y.append(category)

        return cur_string,cur_x,cur_y
    
    @staticmethod
    def zero_pad_to_numpy(data,catagory_size,max_len= None):
        if max_len==None:
            max_len = 0

            for j in range(len(data)):
                if len(data[j])>max_len:
                    max_len = len(data[j])

        new_data = np.zeros((len(data),max_len,catagory_size),dtype=np.int)

        for j in range(len(data)):
            for k in range(len(data[j])):
                new_data[j][k][data[j][k]] = 1

            for k in range(len(data[j]),max_len):
                new_data[j][k][0] = 1
        return new_data
    
    def generate_text_data(self,num_data=40):
        x_data = []
        x_text = []

        y_data = []

        for i in range(num_data):

            cur_string = ""
            cur_x = []
            cur_y = []

            # Add a random amount of spaces
            cur_string,cur_x,cur_y = self.add_spaces(cur_string,cur_x,cur_y,0,2)

            # Add a random date
            cur_string,cur_x,cur_y = self.add_date(cur_string,cur_x,cur_y)

            # Add random amount of spaces
            cur_string,cur_x,cur_y = self.add_spaces(cur_string,cur_x,cur_y,1,2)

            # Add a random description
            cur_string,cur_x,cur_y = self.random_discription(cur_string,cur_x,cur_y,20)

            # Add random amount of spaces
            cur_string,cur_x,cur_y = self.add_spaces(cur_string,cur_x,cur_y,0,2)

            # Add a random rand value
            cur_string,cur_x,cur_y = self.add_rand_value(cur_string,3,cur_x,cur_y)

            # Add random amount of spaces
            cur_string,cur_x,cur_y = self.add_spaces(cur_string,cur_x,cur_y,0,2)

            # Add a random rand value
            cur_string,cur_x,cur_y = self.add_rand_value(cur_string,4,cur_x,cur_y)

            x_text.append(cur_string)
            x_data.append(cur_x)
            y_data.append(cur_y)

        x_data = np.array(x_data)
        y_data = np.array(y_data)

        return x_data, y_data, x_text

# Generate data

In [113]:
np.random.seed(1330)

# Hyper parameters
num_data = 10000

# Generate the data
gen = generator()
x_train,y_train,x_train_text = gen.generate_text_data(num_data)

# Zero pad the data and convert the data into the correct format
input_cat = len(gen.int_to_char)
output_cat = 5
x_train = gen.zero_pad_to_numpy(x_train,input_cat)
y_train = gen.zero_pad_to_numpy(y_train,output_cat)

#print(x_train_text[0])
#print(y_train[0])

char_to_int:  {'q': 16, 'v': 21, 'V': 47, 'H': 33, 'J': 35, 'T': 45, 'm': 12, '5': 57, 'a': 0, ' ': 64, 'b': 1, 'F': 31, 'L': 37, '.': 62, 'K': 36, 'u': 20, 'O': 40, '0': 52, 's': 18, 'd': 3, 'X': 49, 'j': 9, '4': 56, '1': 53, 'i': 8, 'Y': 50, '6': 58, 'N': 39, 'W': 48, '8': 60, '-': 63, 'Q': 42, 'G': 32, 'M': 38, 'p': 15, '2': 54, 'h': 7, '7': 59, 'B': 27, 'g': 6, 'f': 5, 'x': 23, 'r': 17, 't': 19, 'n': 13, 'k': 10, 'R': 43, 'w': 22, 'c': 2, 'A': 26, 'o': 14, 'z': 25, 'I': 34, '3': 55, 'D': 29, 'l': 11, 'S': 44, 'E': 30, 'e': 4, 'U': 46, 'P': 41, 'C': 28, 'Z': 51, '9': 61, 'y': 24}
int_to_char:  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.', '-', ' ']


# Create the model

In [114]:
def create_model(input_dim,output_cat):
    model=Sequential()
    lstm_par = LSTM(units = 50, return_sequences = True)
    model.add(Bidirectional(lstm_par,input_shape=(None, input_cat)))
    model.add(Dense(units = 50))
    model.add(Dense(activation="softmax", units=output_cat))
    model.compile(loss = "mse",optimizer='adam', metrics=['accuracy'])
    return model

In [115]:
# Create the model
input_cat = len(gen.int_to_char)
output_cat = 5
model = create_model(input_cat,output_cat)

# Fit the model

In [116]:
model.fit(x_train, y_train, epochs = 10, batch_size = 400)

Epoch 1/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.1332 - acc: 0.4888
Epoch 2/10
10000/10000 [==============================] - 9s 942us/step - loss: 0.0748 - acc: 0.7460
Epoch 3/10
10000/10000 [==============================] - 9s 945us/step - loss: 0.0308 - acc: 0.9169
Epoch 4/10
10000/10000 [==============================] - 9s 942us/step - loss: 0.0191 - acc: 0.9525
Epoch 5/10
10000/10000 [==============================] - 9s 941us/step - loss: 0.0118 - acc: 0.9736
Epoch 6/10
10000/10000 [==============================] - 10s 964us/step - loss: 0.0044 - acc: 0.9937
Epoch 7/10
10000/10000 [==============================] - 9s 949us/step - loss: 0.0022 - acc: 0.9967
Epoch 8/10
10000/10000 [==============================] - 9s 945us/step - loss: 0.0015 - acc: 0.9974
Epoch 9/10
10000/10000 [==============================] - 10s 963us/step - loss: 0.0012 - acc: 0.9977
Epoch 10/10
10000/10000 [==============================] - 9s 948us/step - loss: 9.7941e-0

# Test the model on new test data

In [117]:
# Generate the data
x_test,y_test,x_test_text = gen.generate_text_data(1000)
input_cat = len(gen.int_to_char)
output_cat = 5

# Zero pad the data and convert the data into the correct format
x_test = gen.zero_pad_to_numpy(x_test,input_cat,max_len= 44)

# Do a prediction with the model
test_index = 100
print("Input: ",x_test_text[test_index])
print("Expected output: ",y_test[test_index])
reshaped_data = x_test[test_index].reshape((1,44,65))

output = model.predict(reshaped_data)
print("Model output: ",np.argmax(output[0],axis=1))

Input:   8-27 GtZK7D9HKp42nsv-u7rzR906.8 R400.79
Expected output:  [0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 4, 4, 4, 4, 4, 4, 4]
Model output:  [0 1 1 1 1 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 0 4 4 4 4
 4 4 4 0 0 0 0]
